<a href="https://colab.research.google.com/github/pcashman21/feral-cat-census/blob/main/src/notebooks/image_pair_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Generate training pairs for a siamese neural network to detect identical cat faces.  Based on https://pyimagesearch.com/2020/11/23/building-image-pairs-for-siamese-networks-with-python/.

In [1]:
# import the necessary packages
from imutils import build_montages
import numpy as np
import pandas as pd
import cv2
import os
import zipfile
import matplotlib.pyplot as plt

In [2]:
def make_pairs(images, labels):
	# initialize two empty lists to hold the (image, image) pairs and
	# labels to indicate if a pair is positive or negative
	pairImages = []
	pairLabels = []
  # calculate the total number of classes present in the dataset
	# and then build a list of indexes for each class label that
	# provides the indexes for all examples with a given label
	numClasses = len(np.unique(labels))
	idx = [np.where(labels == i)[0] for i in range(0, numClasses)]
  # loop over all images
	for idxA in range(len(images)):
		# grab the current image and label belonging to the current
		# iteration
		currentImage = images[idxA]
		label = labels[idxA]
		# randomly pick an image that belongs to the *same* class
		# label
		idxB = np.random.choice(idx[label])
		posImage = images[idxB]
		# prepare a positive pair and update the images and labels
		# lists, respectively
		pairImages.append([currentImage, posImage])
		pairLabels.append([1])
    # grab the indices for each of the class labels *not* equal to
		# the current label and randomly pick an image corresponding
		# to a label *not* equal to the current label
		negIdx = np.where(labels != label)[0]
		negImage = images[np.random.choice(negIdx)]
		# prepare a negative pair of images and update our lists
		pairImages.append([currentImage, negImage])
		pairLabels.append([0])
	# return a 2-tuple of our image pairs and labels
	return (np.array(pairImages), np.array(pairLabels))

In [8]:
IMG_SIZE = 244
NUMBER_OF_TRANSFORMS = 50
IMAGE_FOLDER = 'cat-face-transformed'
PATH_TO_READ = os.path.join('/content/content', IMAGE_FOLDER)

In [5]:
# Be sure to upload `cat-face-transformed.zip` before running this cell.
zip_archive = zipfile.ZipFile(PATH_TO_READ + '.zip', 'r')
zip_archive.extractall('/content')
zip_archive.close()

In [9]:
df = pd.read_csv(os.path.join('/content', IMAGE_FOLDER + '.csv'))
# df['image'] = 0
# for i in range(len(df)):
#  df['image'].iloc[i] = plt.imread(os.path.join('/content', IMAGE_FOLDER, df['filename'].iloc[i]))
# df.head()
image_path = os.path.join('/content', IMAGE_FOLDER, df['filename'].iloc[0])
print(image_path)
plt.imshow(plt.imread(os.path.join('/content', IMAGE_FOLDER, df['filename'].iloc[0])))



/content/content/cat-face-transformed/img91.jpg


FileNotFoundError: ignored

In [ ]:
def load_train_test_split(df):
  """
  Splits the data into training and testing sets.

  df is assumed to have the following structure:
  1. df.columns = [filename, label]
  2. Each filename of an original (not transformed) image is of the form 'imgNNN.jpg'
  3. df.label is a string which is the numeric part of the filename (as a string, not as an int)
  4. There are M transforms of each original.
  """
  # split the data into training and testing splits using 75% of
	# the data for training and the remaining 25% for testing
  labels = df['label'].unique()
  train_df = pd.DataFrame(columns=df.columns)
  test_df = pd.DataFrame(columns=df.columns)
  for i in range(len(labels)):
    label = labels[i]
    label_index = df.groupby('label').get_group(label).index.to_list
    train_index = label_index[:int(len(label_index) * 0.75)]
    test_index = label_index[int(len(label_index) * 0.75):]
    train_df = train_df.append(df.iloc[train_index])
    test_df = test_df.append(df.iloc[test_index])
  return [train_df['filename'], train_df['label']], [test_df['filename'], test_df['label']]









In [ ]:
# load MNIST dataset and scale the pixel values to the range of [0, 1]
# print("[INFO] loading MNIST dataset...")
#(trainX, trainY), (testX, testY) = mnist.load_data()

# load cat-face-transformed dataset, consisting of 244x244 images of cats.
# The dataset has about 100 original images of cats amd 50 transforms of
# each original.
print("[INFO] loading cat-face-transformed dataset...")
(trainX, trainY), (testX, testY) = load_train_test_split(df)



In [ ]:
# build the positive and negative image pairs
print("[INFO] preparing positive and negative pairs...")
(pairTrain, labelTrain) = make_pairs(trainX, trainY)
(pairTest, labelTest) = make_pairs(testX, testY)
# initialize the list of images that will be used when building our
# montage
images = []
# loop over a sample of our training pairs
for i in np.random.choice(np.arange(0, len(pairTrain)), size=(49,)):
	# grab the current image pair and label
	imageA = pairTrain[i][0]
	imageB = pairTrain[i][1]
	label = labelTrain[i]
	# to make it easier to visualize the pairs and their positive or
	# negative annotations, we're going to "pad" the pair with four
	# pixels along the top, bottom, and right borders, respectively
	output = np.zeros((36, 60), dtype="uint8")
	pair = np.hstack([imageA, imageB])
	output[4:32, 0:56] = pair
	# set the text label for the pair along with what color we are
	# going to draw the pair in (green for a "positive" pair and
	# red for a "negative" pair)
	text = "neg" if label[0] == 0 else "pos"
	color = (0, 0, 255) if label[0] == 0 else (0, 255, 0)
	# create a 3-channel RGB image from the grayscale pair, resize
	# it from 60x36 to 96x51 (so we can better see it), and then
	# draw what type of pair it is on the image
	vis = cv2.merge([output] * 3)
	vis = cv2.resize(vis, (96, 51), interpolation=cv2.INTER_LINEAR)
	cv2.putText(vis, text, (2, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.75,
		color, 2)
	# add the pair visualization to our list of output images
	images.append(vis)
# construct the montage for the images
montage = build_montages(images, (96, 51), (7, 7))[0]
# show the output montage
cv2.imshow("Siamese Image Pairs", montage)
cv2.waitKey(0)